In [1]:
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, SimpleRNN, RNN, LSTM, GRU, SpatialDropout1D, Dropout

In [2]:
import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt
%matplotlib tk

In [3]:
import os, os.path, sys

In [4]:
path = r"C:\Users\mata2\Desktop\master\podaci\0k\y"

In [5]:
dir = os.listdir(path)

Ucitavanje fajlova

In [6]:
file_names = []
for file in dir:
    file_names.append(file)

In [7]:
percent_train = 0.8

In [8]:
train_files_names = file_names[:int(percent_train*len(file_names))]
test_file_names = file_names[int(percent_train*len(file_names))::]

Ucitavanje zadate trajektorije

In [9]:
y_command = np.asarray(pd.read_csv(r"C:\Users\mata2\Desktop\master\podaci\yCmd.txt",header=None).astype('int'))

Konkatenacija fajlova koji se posle pravilno rasporedjuju na adekvatne train/test skupove

In [10]:
train_ = np.concatenate([pd.read_csv(path + "/" + file, header=None)[:61000].astype('int')
                              for file in train_files_names], axis=0)
test_ = np.concatenate([pd.read_csv(path + "/" + file, header=None)[:61000].astype('int')
                              for file in test_file_names], axis=0)

Prethodno konkatenizovani podaci se adekvatno dele kako bi se izbeglo fiktivno stanje (stanje pri kraju rada i pocetku novog ako su konkatenizovani podaci)

In [11]:
for j in range(len(train_files_names)):
    exec(f"razlika_trening_{j} = []")
    for i in range(61000):
        exec(f"razlika_trening_{j}.append(y_command[i] - train_[j*61000+i])")
    exec(f'razlika_trening_{j} = np.asarray(razlika_trening_{j})')        

In [13]:
for j in range(len(test_file_names)):
    exec(f"razlika_test_{j} = []")
    for i in range(61000):
        exec(f"razlika_test_{j}.append(y_command[i] - test_[j*61000+i])")
    exec(f'razlika_test_{j} = np.asarray(razlika_test_{j})')    

Funkcija koja deli podatke na "sliding window"

In [12]:
def create_timeseries_split(data, time_steps):

  # Get the number of samples in the data
  num_samples = len(data) - time_steps

  # Create empty arrays to store features and target values
  x_train = np.zeros((num_samples, time_steps, 1))
  y_train = np.zeros((num_samples, 1))

  # Loop through the data and create features and target values
  for i in range(num_samples):
    # Extract a slice of data for the current feature
    x_train[i] = data[i:i+time_steps, :]

    # The target value is the next value after the feature
    y_train[i] = data[i+time_steps, 0]

  return x_train, y_train

Buffer

In [14]:
time_steps = 10

Deljenje podataka na train i test skup

In [15]:
for i in range(len(train_files_names)):
    exec(f'train_X_{i}, train_y_{i} = create_timeseries_split(razlika_trening_{i}, time_steps)')

for i in range(len(test_file_names)):
    exec(f'test_X_{i}, test_y_{i} = create_timeseries_split(razlika_test_{i}, time_steps)')

train_X = np.asarray(train_X_0)
for i in range(1,len(train_files_names)):
    exec(f'train_X = np.append(train_X, train_X_{i}, axis=0)')

train_y = np.asarray(train_y_0)
for i in range(1,len(train_files_names)):
    exec(f'train_y = np.append(train_y, train_y_{i}, axis=0)')

test_X = np.asarray(test_X_0)
for i in range(1,len(test_file_names)):
    exec(f'test_X = np.append(test_X, test_X_{i}, axis=0)')

test_y = np.asarray(test_y_0)
for i in range(1,len(test_file_names)):
    exec(f'test_y = np.append(test_y, test_y_{i}, axis=0)')

In [16]:
input_shape = (time_steps,1)

In [24]:
keras.utils.set_random_seed(7)
optimizerMSE = keras.optimizers.Adam(learning_rate=0.0001)

modelMSE = Sequential()
modelMSE.add(GRU(units = 8, input_shape = input_shape, activation = keras.layers.LeakyReLU()))
modelMSE.add(Dense(units = 8, activation=keras.layers.LeakyReLU()))
modelMSE.add(Dense(units = 16, activation=keras.layers.LeakyReLU()))
modelMSE.add(Dense(units = 1, activation=keras.layers.LeakyReLU(), bias_initializer='zeros', kernel_initializer='normal'))
modelMSE.compile(loss='mean_squared_error', optimizer = optimizerMSE)

In [25]:
historyMSE = modelMSE.fit(train_X, train_y, epochs=100, batch_size = 256, shuffle=False, verbose = 2)

Epoch 1/100

5718/5718 - 22s - loss: 22.1633 - 22s/epoch - 4ms/step
Epoch 2/100
5718/5718 - 19s - loss: 4.1340 - 19s/epoch - 3ms/step
Epoch 3/100
5718/5718 - 19s - loss: 2.8928 - 19s/epoch - 3ms/step
Epoch 4/100
5718/5718 - 19s - loss: 2.5046 - 19s/epoch - 3ms/step
Epoch 5/100
5718/5718 - 19s - loss: 2.3370 - 19s/epoch - 3ms/step
Epoch 6/100
5718/5718 - 18s - loss: 2.2315 - 18s/epoch - 3ms/step
Epoch 7/100
5718/5718 - 19s - loss: 2.1474 - 19s/epoch - 3ms/step
Epoch 8/100
5718/5718 - 20s - loss: 2.0741 - 20s/epoch - 3ms/step
Epoch 9/100
5718/5718 - 19s - loss: 2.0089 - 19s/epoch - 3ms/step
Epoch 10/100
5718/5718 - 19s - loss: 1.9547 - 19s/epoch - 3ms/step
Epoch 11/100
5718/5718 - 19s - loss: 1.9052 - 19s/epoch - 3ms/step
Epoch 12/100
5718/5718 - 19s - loss: 1.8652 - 19s/epoch - 3ms/step
Epoch 13/100
5718/5718 - 19s - loss: 1.8311 - 19s/epoch - 3ms/step
Epoch 14/100
5718/5718 - 19s - loss: 1.7989 - 19s/epoch - 3ms/step
Epoch 15/100
5718/5718 - 19s - loss: 1.7687 - 19s/epoch - 3ms/step
Ep

In [26]:
train_scores = modelMSE.evaluate(test_X, test_y)

11436/11436 [==============================] - 35s 3ms/step - loss: 1.2571


In [21]:
#train_predict_MSE = modelMSE.predict(train_X)
test_predict_MSE = modelMSE.predict(test_X)

11436/11436 [==============================] - 21s 2ms/step


In [32]:
modelMSE.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 8)                 264       
                                                                 
 dense (Dense)               (None, 8)                 72        
                                                                 
 dense_1 (Dense)             (None, 16)                144       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 497 (1.94 KB)
Trainable params: 497 (1.94 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Ucitavamo standardne devijacije i srednje vrednosti

In [17]:
std_dev = []
with open(r"C:\Users\mata2\Desktop\master\Axis-projection-RNN\odstupanja\0k Y\Y_std_dev.txt", 'r') as lines:
    for line in lines:
        std_dev.append(float(line.strip('\n')))

In [18]:
sr_vrednost = []
with open(r"C:\Users\mata2\Desktop\master\Axis-projection-RNN\odstupanja\0k Y\Y_sr_vrednost.txt", 'r') as lines:
    for line in lines:
        sr_vrednost.append(float(line.strip('\n')))

In [29]:
mean_1std = 0
for j in range(len(test_file_names)):
    for i in range(int(len(test_predict_MSE)/len(test_file_names))):
        if sr_vrednost[i+time_steps] - std_dev[i+time_steps] <= test_predict_MSE[int(len(test_predict_MSE)/len(test_file_names))*j + i] <= sr_vrednost[i+time_steps] + std_dev[i+time_steps]:
            mean_1std += 1
        #else:
            #print(sr_vrednost[i+8], std_dev[i+8], test_predict_MSE[60992*j + i], 'index:',j*60992 + i)
mean_1std/len(test_predict_MSE)

0.6589386238181123

Mean 3 sigma

In [30]:
i = time_steps
mean_3std = 0
for j in range(len(test_predict_MSE)):
    if i%61000 == 0:
        i = time_steps
    if sr_vrednost[i] - 3*std_dev[i] <= test_predict_MSE[j] <= sr_vrednost[i] + 3*std_dev[i]:
            mean_3std += 1
    i += 1
mean_3std/len(test_predict_MSE)

0.9915395966551894

pravljenje min/max razlika - posle se koriste za plotovanje

In [19]:
min_kord = [float('inf') for x in range(61000)]
max_kord = [float('-inf') for x in range(61000)]

for i in range(61000):    
    for j in range(len(train_files_names)):
        exec(f'current_file = razlika_trening_{j}')
        if current_file[i][0] < min_kord[i]:
            min_kord[i] = current_file[i][0]
        if current_file[i][0] > max_kord[i]:
            max_kord[i] = current_file[i][0]

cuvamo/ucitavamo model

# NA TEST SKUPU - plotovi trajektorija

test_ = realizovana trajektorija

test_predict_MSE = predvidjena RAZLIKA koordinata od y_command

razlika_test_{} = realizovana RAZLIKA od test podataka od y_command - CELA TRAJEKTORIJA

test_y_{} = REALIZOVANA y koordinata od razlika_test_{}

Y_COMMAND - TEST_ = ODSTUPANJE/RAZLIKA_TEST_

Y_COMMAND[TIME_STEPS] + test_predict_MSE[0] = PRVU KOORDINATU TRAJEKTORIJE PREDVIDJENE VREDNOSTI

In [23]:
for j in range(len(test_file_names)):
    exec(f'predvidjena_trajektorija_{j} = []')
    for i in range(61000 - time_steps):
        exec(f'predvidjena_trajektorija_{j}.append(y_command[i + time_steps] + test_predict_MSE[j*(61000-time_steps) + i])')

In [22]:
for j in range(len(test_file_names)):
    exec(f'realizovana_trajektorija_{j} = []')
    for i in range(61000 - time_steps):
        exec(f'realizovana_trajektorija_{j}.append(test_[j*61000 + i])')

In [24]:
command_length = len(y_command)

In [89]:
plt.figure(figsize=(12,7))
plt.subplot(211)
plt.axvline(x = 4100, color = 'b', label = 'axvline - full height')
plt.plot(y_command[time_steps::],'r')
plt.plot(predvidjena_trajektorija_0,'b')
plt.plot(realizovana_trajektorija_0[time_steps::], 'g')
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Y осе машине (μm)', fontsize=12)
plt.show()

plt.subplot(212)
plt.axvline(x = 4100, color = 'b', label = 'axvline - full height')
plt.plot(y_command[time_steps::],'r')
plt.plot(predvidjena_trajektorija_0,'b')
plt.plot(realizovana_trajektorija_0[time_steps::], 'g')
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Y осе машине (μm)', fontsize=12)
plt.legend(["_HiddenLabel","задата трајекторија", "предвиђена трајекторија 1", "реализована трајекторија 1"])
plt.show()

C:\Users\mata2\AppData\Local\Temp\ipykernel_18184\1933976141.py:18: UserWarning: The label '_HiddenLabel' of <matplotlib.lines.Line2D object at 0x00000232C35DEDD0> starts with '_'. It is thus excluded from the legend.
  plt.legend(["_HiddenLabel","задата трајекторија", "предвиђена трајекторија 1", "реализована трајекторија 1"])


In [90]:
plt.figure(figsize=(12,7))
plt.subplot(211)
plt.axvline(x = 12690, color = 'b', label = 'axvline - full height')
plt.plot(y_command[time_steps::],'r')
plt.plot(predvidjena_trajektorija_1,'b')
plt.plot(realizovana_trajektorija_1[time_steps::], 'g')
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Y осе машине (μm)', fontsize=12)
plt.show()

plt.subplot(212)
plt.axvline(x = 12690, color = 'b', label = 'axvline - full height')
plt.plot(y_command[time_steps::],'r')
plt.plot(predvidjena_trajektorija_1,'b')
plt.plot(realizovana_trajektorija_1[time_steps::], 'g')
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Y осе машине (μm)', fontsize=12)
plt.legend(["_HiddenLabel","задата трајекторија", "предвиђена трајекторија 2", "реализована трајекторија 2"])
plt.show()

C:\Users\mata2\AppData\Local\Temp\ipykernel_18184\3260072544.py:18: UserWarning: The label '_HiddenLabel' of <matplotlib.lines.Line2D object at 0x00000232C66A3D90> starts with '_'. It is thus excluded from the legend.
  plt.legend(["_HiddenLabel","задата трајекторија", "предвиђена трајекторија 2", "реализована трајекторија 2"])


In [91]:
plt.figure(figsize=(12,7))
plt.subplot(211)
plt.axvline(x = 15000, color = 'b', label = 'axvline - full height')
plt.plot(y_command[time_steps::],'r')
plt.plot(predvidjena_trajektorija_2,'b')
plt.plot(realizovana_trajektorija_2[time_steps::], 'g')
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Y осе машине (μm)', fontsize=12)
plt.show()

plt.subplot(212)
plt.axvline(x = 15000, color = 'b', label = 'axvline - full height')
plt.plot(y_command[time_steps::],'r')
plt.plot(predvidjena_trajektorija_2,'b')
plt.plot(realizovana_trajektorija_2[time_steps::], 'g')
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Y осе машине (μm)', fontsize=12)
plt.legend(["вертикална плава линија на индексу 15000","задата трајекторија", "предвиђена трајекторија 3", "реализована трајекторија 3"])
plt.show()

In [92]:
plt.figure(figsize=(12,7))
plt.subplot(211)
plt.axvline(x = 34250, color = 'b', label = 'axvline - full height')
plt.plot(y_command[time_steps::],'r')
plt.plot(predvidjena_trajektorija_3,'b')
plt.plot(realizovana_trajektorija_3[time_steps::], 'g')
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Y осе машине (μm)', fontsize=12)
plt.show()

plt.subplot(212)
plt.axvline(x = 34250, color = 'b', label = 'axvline - full height')
plt.plot(y_command[time_steps::],'r')
plt.plot(predvidjena_trajektorija_3,'b')
plt.plot(realizovana_trajektorija_3[time_steps::], 'g')
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Y осе машине (μm)', fontsize=12)
plt.legend(["_Hidden","задата трајекторија", "предвиђена трајекторија 4", "реализована трајекторија 4"])
plt.show()

C:\Users\mata2\AppData\Local\Temp\ipykernel_18184\1460834486.py:18: UserWarning: The label '_Hidden' of <matplotlib.lines.Line2D object at 0x00000232DE596550> starts with '_'. It is thus excluded from the legend.
  plt.legend(["_Hidden","задата трајекторија", "предвиђена трајекторија 4", "реализована трајекторија 4"])


In [93]:
plt.figure(figsize=(12,7))
plt.subplot(211)
plt.axvline(x = 55900, color = 'b', label = 'axvline - full height')
plt.plot(y_command[time_steps::],'r')
plt.plot(predvidjena_trajektorija_4,'b')
plt.plot(realizovana_trajektorija_4[time_steps::], 'g')
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Y осе машине (μm)', fontsize=12)
plt.show()

plt.subplot(212)
plt.axvline(x = 55900, color = 'b', label = 'axvline - full height')
plt.plot(y_command[time_steps::],'r')
plt.plot(predvidjena_trajektorija_4,'b')
plt.plot(realizovana_trajektorija_4[time_steps::], 'g')
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Y осе машине (μm)', fontsize=12)
plt.legend(["_Hidden","задата трајекторија", "предвиђена трајекторија 5", "реализована трајекторија 5"])
plt.show()

C:\Users\mata2\AppData\Local\Temp\ipykernel_18184\3836706913.py:18: UserWarning: The label '_Hidden' of <matplotlib.lines.Line2D object at 0x00000232C9F4F510> starts with '_'. It is thus excluded from the legend.
  plt.legend(["_Hidden","задата трајекторија", "предвиђена трајекторија 5", "реализована трајекторија 5"])


In [94]:
plt.figure(figsize=(12,7))
plt.subplot(211)
plt.axvline(x = 43150, color = 'b', label = 'axvline - full height')
plt.plot(y_command[time_steps::],'r')
plt.plot(predvidjena_trajektorija_5,'b')
plt.plot(realizovana_trajektorija_5[time_steps::], 'g')
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Y осе машине (μm)', fontsize=12)
plt.show()

plt.subplot(212)
plt.axvline(x = 43150, color = 'b', label = 'axvline - full height')
plt.plot(y_command[time_steps::],'r')
plt.plot(predvidjena_trajektorija_5,'b')
plt.plot(realizovana_trajektorija_5[time_steps::], 'g')
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Y осе машине (μm)', fontsize=12)
plt.legend(["_Hidden","задата трајекторија", "предвиђена трајекторија 6", "реализована трајекторија 6"])
plt.show()

C:\Users\mata2\AppData\Local\Temp\ipykernel_18184\81646673.py:18: UserWarning: The label '_Hidden' of <matplotlib.lines.Line2D object at 0x00000232D2034950> starts with '_'. It is thus excluded from the legend.
  plt.legend(["_Hidden","задата трајекторија", "предвиђена трајекторија 6", "реализована трајекторија 6"])


# NA TEST SKUPU - 3 sigma (upadanje u pojas) plotovi

In [25]:
tri_sigma_lower = []
tri_sigma_upper = []

for i in range(len(y_command)):
    tri_sigma_lower.append(sr_vrednost[i] - 3*std_dev[i])
    tri_sigma_upper.append(sr_vrednost[i] + 3*std_dev[i])

In [26]:
for j in range(len(test_file_names)):
    exec(f'trajektorija_odstupanja_{j} = []')
    for i in range(61000 - time_steps):
        exec(f'trajektorija_odstupanja_{j}.append(test_predict_MSE[j*(61000-time_steps) + i])')

In [28]:
plt.figure(figsize=(11,7))
plt.plot(tri_sigma_lower[time_steps:], color= 'b')
plt.plot(tri_sigma_upper[time_steps:], color = 'r')
plt.plot(trajektorija_odstupanja_0, color = 'g')
plt.xlabel('време (ms)', fontsize=18)
plt.ylabel('одступање (μm)', fontsize=18)
plt.legend(['три сигма доња граница',"три сигма горња граница", "одступање трајекторије 1"],loc = 'best')
plt.show()

In [30]:
trajektorija = 1
i = time_steps
mean_3std = 0
for j in range(len(test_predict_MSE)):
    if i%len(y_command) == 0:
        print("trajektorija ",trajektorija, " sadrzi ", mean_3std , " tacaka", ", procenat tri sigma iznosi: ",mean_3std/61000)
        exec(f"trajektorija_{trajektorija} = mean_3std")
        trajektorija += 1
        mean_3std = 0
        i = time_steps
    if sr_vrednost[i] - 3*std_dev[i] <= test_predict_MSE[j] <= sr_vrednost[i] + 3*std_dev[i]:
            mean_3std += 1
    i += 1
trajektorija_6 = mean_3std
print("trajektorija ",trajektorija, " sadrzi ", mean_3std , " tacaka", ", procenat tri sigma iznosi: ",mean_3std/61000)

trajektorija  1  sadrzi  60600  tacaka , procenat tri sigma iznosi:  0.9934426229508196
trajektorija  2  sadrzi  60386  tacaka , procenat tri sigma iznosi:  0.9899344262295082
trajektorija  3  sadrzi  60178  tacaka , procenat tri sigma iznosi:  0.9865245901639345
trajektorija  4  sadrzi  60453  tacaka , procenat tri sigma iznosi:  0.9910327868852459
trajektorija  5  sadrzi  60667  tacaka , procenat tri sigma iznosi:  0.9945409836065574
trajektorija  6  sadrzi  60560  tacaka , procenat tri sigma iznosi:  0.9927868852459016


In [31]:
plt.figure(figsize=(11,7))
for i in range(len(trajektorija_odstupanja_0)):
    if tri_sigma_lower[time_steps + i] <= trajektorija_odstupanja_0[i] <= tri_sigma_upper[time_steps + i]:
        plt.plot(i, trajektorija_odstupanja_0[i], color='black', marker = 'o', markersize = 2)
    else:
        plt.plot(i, trajektorija_odstupanja_0[i], color='red', marker = 'o', markersize = 2)
plt.title('Количина тачака која упада у три сигма појас износи: {}%'.format(round((trajektorija_1/61000)*100,2)) , fontsize=18)
plt.xlabel('време (ms)', fontsize=18)
plt.ylabel('одступање (μm)', fontsize=18)
plt.show()

In [32]:
plt.figure(figsize=(11,7))
for i in range(len(trajektorija_odstupanja_1)):
    if tri_sigma_lower[time_steps + i] <= trajektorija_odstupanja_1[i] <= tri_sigma_upper[time_steps + i]:
        plt.plot(i, trajektorija_odstupanja_1[i], color='black', marker = 'o', markersize = 2)
    else:
        plt.plot(i, trajektorija_odstupanja_1[i], color='red', marker = 'o', markersize = 2)
plt.title('Количина тачака која упада у три сигма појас износи: {}%'.format(round((trajektorija_2/61000)*100,2)) , fontsize=18)
plt.xlabel('време (ms)', fontsize=18)
plt.ylabel('одступање (μm)', fontsize=18)
plt.show()

In [33]:
plt.figure(figsize=(11,7))
for i in range(len(trajektorija_odstupanja_2)):
    if tri_sigma_lower[time_steps + i] <= trajektorija_odstupanja_2[i] <= tri_sigma_upper[time_steps + i]:
        plt.plot(i, trajektorija_odstupanja_2[i], color='black', marker = 'o', markersize = 2)
    else:
        plt.plot(i, trajektorija_odstupanja_2[i], color='red', marker = 'o', markersize = 2)
plt.title('Количина тачака која упада у три сигма појас износи: {}%'.format(round((trajektorija_3/61000)*100,2)) , fontsize=18)
plt.xlabel('време (ms)', fontsize=18)
plt.ylabel('одступање (μm)', fontsize=18)
plt.show()

In [34]:
plt.figure(figsize=(11,7))
for i in range(len(trajektorija_odstupanja_3)):
    if tri_sigma_lower[time_steps + i] <= trajektorija_odstupanja_3[i] <= tri_sigma_upper[time_steps + i]:
        plt.plot(i, trajektorija_odstupanja_3[i], color='black', marker = 'o', markersize = 2)
    else:
        plt.plot(i, trajektorija_odstupanja_3[i], color='red', marker = 'o', markersize = 2)
plt.title('Количина тачака која упада у три сигма појас износи: {}%'.format(round((trajektorija_4/61000)*100,2)) , fontsize=18)
plt.xlabel('време (ms)', fontsize=18)
plt.ylabel('одступање (μm)', fontsize=18)
plt.show()

In [35]:
plt.figure(figsize=(11,7))
for i in range(len(trajektorija_odstupanja_4)):
    if tri_sigma_lower[time_steps + i] <= trajektorija_odstupanja_4[i] <= tri_sigma_upper[time_steps + i]:
        plt.plot(i, trajektorija_odstupanja_4[i], color='black', marker = 'o', markersize = 2)
    else:
        plt.plot(i, trajektorija_odstupanja_4[i], color='red', marker = 'o', markersize = 2)
plt.title('Количина тачака која упада у три сигма појас износи: {}%'.format(round((trajektorija_5/61000)*100,2)) , fontsize=18)
plt.xlabel('време (ms)', fontsize=18)
plt.ylabel('одступање (μm)', fontsize=18)
plt.show()

In [36]:
plt.figure(figsize=(11,7))
for i in range(len(trajektorija_odstupanja_5)):
    if tri_sigma_lower[time_steps + i] <= trajektorija_odstupanja_5[i] <= tri_sigma_upper[time_steps + i]:
        plt.plot(i, trajektorija_odstupanja_5[i], color='black', marker = 'o', markersize = 2)
    else:
        plt.plot(i, trajektorija_odstupanja_5[i], color='red', marker = 'o', markersize = 2)
plt.title('Количина тачака која упада у три сигма појас износи: {}%'.format(round((trajektorija_6/61000)*100,2)) , fontsize=18)
plt.xlabel('време (ms)', fontsize=18)
plt.ylabel('одступање (μm)', fontsize=18)
plt.show()

# NA TEST SKUPU - min/max plotovi

MIN MAX TRAJEKTORIJE

In [25]:
min_trajektorija = []
max_trajektorija = []
for i in range(61000):
    min_trajektorija.append(y_command[i] + min_kord[i])
    max_trajektorija.append(y_command[i] + max_kord[i])

In [26]:
trajektorija_koord = 1
i = time_steps
mean_3std_koord = 0
for j in range(len(test_predict_MSE)):
    if i%len(y_command) == 0:
        print("trajektorija ",trajektorija_koord, " sadrzi ", mean_3std_koord , " tacaka", ", procenat tri sigma iznosi: ",round(mean_3std_koord/61000*100,2), "%")
        exec(f"trajektorija_koord_{trajektorija_koord} = mean_3std_koord")
        trajektorija_koord += 1
        mean_3std_koord = 0
        i = time_steps
    if min_kord[i] <= (test_predict_MSE[j]) <= max_kord[i]:
            mean_3std_koord += 1
    i += 1
trajektorija_koord_6 = mean_3std_koord
print("trajektorija ",trajektorija_koord, " sadrzi ", mean_3std_koord , " tacaka", ", procenat tri sigma iznosi: ",round(mean_3std_koord/61000*100,2),"%")

trajektorija  1  sadrzi  58125  tacaka , procenat tri sigma iznosi:  95.29 %
trajektorija  2  sadrzi  51879  tacaka , procenat tri sigma iznosi:  85.05 %
trajektorija  3  sadrzi  56354  tacaka , procenat tri sigma iznosi:  92.38 %
trajektorija  4  sadrzi  57087  tacaka , procenat tri sigma iznosi:  93.59 %
trajektorija  5  sadrzi  52260  tacaka , procenat tri sigma iznosi:  85.67 %
trajektorija  6  sadrzi  56427  tacaka , procenat tri sigma iznosi:  92.5 %


In [41]:
i = time_steps
plt.figure(figsize=(11,7))
for j in range(len(predvidjena_trajektorija_0)):
    if i%61000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_0[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_0[j], color='black', marker='o', markersize=2)
    else:
        plt.plot(i, predvidjena_trajektorija_0[j], color='red',marker='o',markersize=2)
    i += 1
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Y осе машине (μm)', fontsize=12)
plt.title('Количина тачака која упада у мин макс појас износи: {}%'.format(round((trajektorija_koord_1/61000)*100,2)) , fontsize=18)
plt.show()

In [42]:
i = time_steps
plt.figure(figsize=(11,7))
for j in range(len(predvidjena_trajektorija_1)):
    if i%61000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_1[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_1[j], color='black', marker='o', markersize=2)
    else:
        plt.plot(i, predvidjena_trajektorija_1[j], color='red',marker='o',markersize=2)
    i += 1
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Y осе машине (μm)', fontsize=12)
plt.title('Количина тачака која упада у мин макс појас износи: {}%'.format(round((trajektorija_koord_2/61000)*100,2)) , fontsize=18)
plt.show()

In [43]:
i = time_steps
plt.figure(figsize=(11,7))
for j in range(len(predvidjena_trajektorija_2)):
    if i%61000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_2[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_2[j], color='black', marker='o', markersize=2)
    else:
        plt.plot(i, predvidjena_trajektorija_2[j], color='red',marker='o',markersize=2)
    i += 1
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Y осе машине (μm)', fontsize=12)
plt.title('Количина тачака која упада у мин макс појас износи: {}%'.format(round((trajektorija_koord_3/61000)*100,2)) , fontsize=18)
plt.show()

In [27]:
i = time_steps
plt.figure(figsize=(11,7))
for j in range(len(predvidjena_trajektorija_3)):
    if i%61000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_3[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_3[j], color='black', marker='o', markersize=2)
    else:
        plt.plot(i, predvidjena_trajektorija_3[j], color='red',marker='o',markersize=2)
    i += 1
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Y осе машине (μm)', fontsize=12)
plt.title('Количина тачака која упада у мин макс појас износи: {}%'.format(round((trajektorija_koord_4/61000)*100,2)) , fontsize=18)
plt.show()

In [28]:
i = time_steps
plt.figure(figsize=(11,7))
for j in range(len(predvidjena_trajektorija_4)):
    if i%61000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_4[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_4[j], color='black', marker='o', markersize=2)
    else:
        plt.plot(i, predvidjena_trajektorija_4[j], color='red',marker='o',markersize=2)
    i += 1
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Y осе машине (μm)', fontsize=12)
plt.title('Количина тачака која упада у мин макс појас износи: {}%'.format(round((trajektorija_koord_5/61000)*100,2)) , fontsize=18)
plt.show()

In [29]:
i = time_steps
plt.figure(figsize=(11,7))
for j in range(len(predvidjena_trajektorija_0)):
    if i%61000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_0[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_0[j], color='black', marker='o', markersize=2)
    else:
        plt.plot(i, predvidjena_trajektorija_0[j], color='red',marker='o',markersize=2)
    i += 1
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Y осе машине (μm)', fontsize=12)
plt.title('Количина тачака која упада у мин макс појас износи: {}%'.format(round((trajektorija_koord_6/61000)*100,2)) , fontsize=18)
plt.show()